In [16]:
# Bagian A — DATA & PRA-PROSES
# import pustaka
import kagglehub
import os, numpy as np, pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error

In [17]:
# unduh dataset Kaggle: borapajo/food-choices
path = kagglehub.dataset_download("borapajo/food-choices")
print("Path to dataset files:", path)

# muat CSV utama (dataset ini bernama 'food_coded.csv')
csv_path = os.path.join(path, "food_coded.csv")
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"File tidak ditemukan: {csv_path}. Isi folder: {os.listdir(path)}")

data = pd.read_csv(csv_path)

Using Colab cache for faster access to the 'food-choices' dataset.
Path to dataset files: /kaggle/input/food-choices


In [18]:
#Pilih fitur & target sesuai tugas
X = data[['exercise', 'Gender', 'comfort_food']]
y = data['calories_day']

In [19]:
# Pilih fitur & target sesuai tugas
X = data[['exercise', 'Gender', 'comfort_food']]
y = data['calories_day']

In [20]:
#Pipeline pra-proses: imputasi + scale (numerik), imputasi + onehot (kategorik)
num_tf = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler',  StandardScaler())
])
cat_tf = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot',  OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_tf, num_cols),
        ('cat', cat_tf, cat_cols)
    ],
    remainder='drop'
)

In [22]:
#Bersihkan baris target yang NaN, lalu split train/test
mask = ~y.isna()
X_clean = X.loc[mask]
y_clean = y.loc[mask]

X_train, X_test, y_train, y_test = train_test_split(
    X_clean, y_clean, test_size=0.2, random_state=42
)

In [23]:
# Fit transformer pada train dan transform ke matriks numerik
pre_fit = preprocessor.fit(X_train)
X_train_t = pre_fit.transform(X_train)   # bisa sparse
X_test_t  = pre_fit.transform(X_test)

# Simpan nama fitur pasca-transform (untuk interpretasi koefisien)
feature_names = pre_fit.get_feature_names_out()

In [25]:
# Pastikan ke numpy array padat untuk operasi lin-alg
Xtr = np.asarray(X_train_t.todense()) if hasattr(X_train_t, "todense") else np.asarray(X_train_t)
Xte = np.asarray(X_test_t.todense())  if hasattr(X_test_t,  "todense") else np.asarray(X_test_t)
ytr = np.asarray(y_train)
yte = np.asarray(y_test)

In [27]:
# Bagian B — OLS (LEAST SQUARES)
# Fungsi OLS mengikuti pola 'multiple_linear_regression' di code referensi
import numpy as np

def ols_pinv(inputs, outputs):
    """
    [TUGAS] Hitung koefisien β OLS: β = (X^T X)^(-1) X^T y (pinv untuk stabilitas).
           Juga hitung COD (R^2 %) dan average error (MSE).
    """
    X = np.asarray(inputs); Y = np.asarray(outputs)
    Xt = X.T
    beta = np.linalg.pinv(Xt @ X) @ (Xt @ Y)       # pseudo-inverse
    Y_hat = X @ beta
    Y_mean = Y.mean()
    SST = ((Y - Y_mean) ** 2).sum()
    SSR = ((Y - Y_hat) ** 2).sum()
    COD = (1 - (SSR / SST)) * 100.0               # percent, seperti referensi
    av_err = SSR / len(Y)                          # MSE
    return {'COD': COD, 'coeff': beta, 'error': av_err}

# Fit OLS pada TRAIN
ols_train = ols_pinv(Xtr, ytr)

# Evaluasi OLS pada TEST
yhat_ols_test = Xte @ ols_train['coeff']
r2_ols = r2_score(yte, yhat_ols_test)
mse_ols = mean_squared_error(yte, yhat_ols_test)

print("\n==== OLS (Baseline, Least Squares) ====")
print(f"TRAIN -> COD: {ols_train['COD']:.3f}% | MSE: {ols_train['error']:.6f}")
print(f"TEST  -> R^2 : {r2_ols:.4f} | MSE: {mse_ols:.4f}")

print("\nKoefisien OLS (fitur -> nilai):")
for fname, coef in zip(feature_names, ols_train['coeff']):
    sign = "positif" if coef > 0 else ("negatif" if coef < 0 else "netral")
    print(f"- {fname}: {coef:.4f} ({sign})")



==== OLS (Baseline, Least Squares) ====
TRAIN -> COD: 100.000% | MSE: 0.000000
TEST  -> R^2 : 0.1202 | MSE: 0.3581

Koefisien OLS (fitur -> nilai):
- num__exercise: -0.1385 (negatif)
- cat__Gender_1: 2.9800 (positif)
- cat__Gender_2: 2.9706 (positif)
- cat__comfort_food_Broccoli, spaghetti squash, quinoa, and grilled chicken: 0.1127 (positif)
- cat__comfort_food_Burger: 0.8903 (positif)
: 0.1221 (positif)
- cat__comfort_food_Chicken fingers, pizza : 0.8903 (positif)
- cat__comfort_food_Chilli, soup, pot pie: 0.3538 (positif)
- cat__comfort_food_Chinese food, moes, sponge candy, homemade lasagne : 0.1127 (positif)
- cat__comfort_food_Chinese, chips, cake: 0.1127 (positif)
- cat__comfort_food_Chips sweets popcorn: -0.1097 (negatif)
- cat__comfort_food_Chips, Mac and cheese, pizza, French fries : -0.1191 (negatif)
- cat__comfort_food_Chips, chocolate, ,mozzarella sticks : 0.0200 (positif)
- cat__comfort_food_Chips, ice cream: 0.8903 (positif)
- cat__comfort_food_Chips, ice cream, microwa

In [28]:
# Bagian C — GA FUNCTIONS (disesuaikan dengan referensi)

# Import untuk GA seperti referensi
from random import random, sample, choice
from math import floor
from tqdm import tqdm                  # progress bar seperti referensi

#  Kita samakan istilah 'inputs' & 'outputs' dengan referensi:
inputs  = Xtr           # fitur TRAIN terproses
outputs = ytr           # target TRAIN
individual_size = inputs.shape[1]

# HYPERPARAMETER GA (mirip referensi namun lebih kecil agar efisien)
population_size = 600
selection_size  = floor(0.2 * population_size)
max_generations = 60
probability_of_individual_mutating = 0.2
probability_of_gene_mutating       = 0.25

# Buat individu acak (koefisien): referensi pakai random() di [0,1)
#  Supaya stabil di data nyata, kita geser ke sekitar 0.
def create_individual(individual_size):
    return [(random() - 0.5) for _ in range(individual_size)]

def create_population(individual_size, population_size):
    return [create_individual(individual_size) for _ in range(population_size)]

# Fitness = COD (R^2 %) & error (MSE), mengikuti referensi
from numpy import array, dot, mean

def get_fitness(individual, inputs):
    predicted_outputs = dot(array(inputs), array(individual))
    output_mean = mean(outputs)
    SST = array([(i - output_mean) ** 2 for i in outputs]).sum()
    SSR = array([(i - j) ** 2 for i, j in zip(outputs, predicted_outputs)]).sum()
    COD = (1 - (SSR / SST)) * 100.0
    av_error = (SSR / len(outputs))  # MSE
    return {'COD': COD, 'error': av_error, 'coeff': individual}

def evaluate_population(population):
    fitness_list = [get_fitness(individual, inputs) for individual in tqdm(population)]
    error_list = sorted(fitness_list, key=lambda i: i['error'])
    best_individuals = error_list[: selection_size]
    best_individuals_stash.append(best_individuals[0]['coeff'])
    print('Error: ', best_individuals[0]['error'], 'COD: ', best_individuals[0]['COD'])
    return best_individuals

def crossover(parent_1, parent_2):
    # sama seperti referensi: ambil setengah gen dari parent_1 dan sisanya dari parent_2
    child = {}
    loci = [i for i in range(0, individual_size)]
    loci_1 = sample(loci, floor(0.5 * individual_size))
    loci_2 = [i for i in loci if i not in loci_1]
    chromosome_1 = [[i, parent_1['coeff'][i]] for i in loci_1]
    chromosome_2 = [[i, parent_2['coeff'][i]] for i in loci_2]
    child.update({key: value for (key, value) in chromosome_1})
    child.update({key: value for (key, value) in chromosome_2})
    return [child[i] for i in loci]

def mutate(individual):
    loci = [i for i in range(0, individual_size)]
    no_of_genes_mutated = max(1, floor(probability_of_gene_mutating * individual_size))
    loci_to_mutate = sample(loci, no_of_genes_mutated)
    for locus in loci_to_mutate:
        gene_transform = choice([-1, 1])
        change = gene_transform * random() * 0.5   # langkah lebih kecil biar stabil
        individual[locus] = individual[locus] + change
    return individual

def get_new_generation(selected_individuals):
    parent_pairs = [sample(selected_individuals, 2) for _ in range(population_size)]
    offspring = [crossover(pair[0], pair[1]) for pair in parent_pairs]
    offspring_indices = [i for i in range(population_size)]
    offspring_to_mutate = sample(
        offspring_indices,
        max(1, floor(probability_of_individual_mutating * population_size))
    )
    mutated_offspring = [[i, mutate(offspring[i])] for i in offspring_to_mutate]
    for child in mutated_offspring:
        offspring[child[0]] = child[1]
    return offspring

def multiple_linear_regression(inputs, outputs):
    """
    Versi OLS ala referensi untuk perbandingan 'best_possible'
    """
    X, Y = array(inputs), array(outputs)
    X_t, Y_t = X.transpose(), Y.transpose()
    coeff = dot((np.linalg.pinv((dot(X_t, X)))), (dot(X_t, Y)))
    Y_p = dot(X, coeff)
    Y_mean = mean(Y)
    SST = array([(i - Y_mean) ** 2 for i in Y]).sum()
    SSR = array([(i - j) ** 2 for i, j in zip(Y, Y_p)]).sum()
    COD = (1 - (SSR / SST)) * 100.0
    av_error = (SSR / len(Y))
    return {'COD': COD, 'coeff': coeff, 'error': av_error}

# Kondisi terminasi (COD tinggi atau mencapai max generasi) — sama seperti referensi
def check_termination_condition(best_individual):
    if ((best_individual['COD'] >= 99.5) or (generation_count == max_generations)):
        return True
    else:
        return False

In [29]:
# Bagian D — MAIN LOOP GA (mengikuti pola referensi)

best_possible = multiple_linear_regression(inputs, outputs)  # OLS-TRAIN terbaik teoritis (pembanding)
best_individuals_stash = [create_individual(individual_size)]
initial_population = create_population(individual_size, population_size)
current_population = initial_population
termination = False
generation_count = 0

print("\n==== Genetic Algorithm (Least Squares objective) ====")
while termination is False:
    current_best_individual = get_fitness(best_individuals_stash[-1], inputs)
    print('Generation: ', generation_count)
    best_individuals   = evaluate_population(current_population)
    current_population = get_new_generation(best_individuals)
    termination        = check_termination_condition(current_best_individual)
    generation_count  += 1
else:
    ga_train_best = get_fitness(best_individuals_stash[-1], inputs)
    print("\nBest (TRAIN) GA:", ga_train_best)



==== Genetic Algorithm (Least Squares objective) ====
Generation:  0


100%|██████████| 600/600 [00:00<00:00, 4106.14it/s]

Error:  7.168245009397874 COD:  -1680.955520644768


Generation:  1


100%|██████████| 600/600 [00:00<00:00, 10714.65it/s]


...

Error:  0.009487961747821447 COD:  97.64270922209056
Generation:  59


100%|██████████| 600/600 [00:00<00:00, 9087.00it/s]


Error:  0.009346231287538889 COD:  97.67792225475795
Generation:  60


100%|██████████| 600/600 [00:00<00:00, 10236.17it/s]


Error:  0.009348781863612418 COD:  97.6772885623363

Best (TRAIN) GA: {'COD': np.float64(97.6772885623363), 'error': np.float64(0.009348781863612418), 'coeff': [-0.11686816969580394, 2.829892930405303, 2.8041223262331525, 0.06751047624428208, 1.0083470985693537, 0.3295224159737724, 0.925341946855103, 0.4736551247927159, 0.28347787345540876, 0.25423254007505913, 0.05853178996068886, 0.05620781623175741, 0.15082091057688896, 1.2268746258138075, -0.805933572259111, 0.11159275566890559, 0.22635195050444717, 0.37646755437749907, 0.12703599488300954, 0.29797264254866923, -0.8064503756405204, 0.1145786642959793, 0.2100230476140515, -0.6377974015054639, 0.298184432844811, 0.22116540327110823, -0.6028410221533196, 0.9757293381017345, 0.4980045604940438, 0.2912134126744356, 1.053218163581395, 0.04298848118908438, 0.11646735841586531, 0.14457375193510819, 0.3311806634398571, 1.045833590425958, -0.5305170694502733, 0.16217179515500524, 0.0835817104652593, 0.10282792661519924, -0.8682133948032276, 

In [31]:
# Evaluasi GA pada TEST
beta_ga = np.array(ga_train_best['coeff'])
yhat_ga_test = Xte @ beta_ga
r2_ga = r2_score(yte, yhat_ga_test)
mse_ga = mean_squared_error(yte, yhat_ga_test)

print("\n==== GA (Evaluasi TEST) ====")
print(f"R^2 (test): {r2_ga:.4f} | MSE (test): {mse_ga:.4f}")



==== GA (Evaluasi TEST) ====
R^2 (test): 0.0800 | MSE (test): 0.3745


In [32]:
# Tampilkan 10 fitur dengan |koef| terbesar versi GA (interpretasi)
top_idx = np.argsort(np.abs(beta_ga))[::-1][:10]
print("\nTop-10 fitur (|coef|) versi GA:")
for i in top_idx:
    print(f"- {feature_names[i]}: {beta_ga[i]:.4f}")


Top-10 fitur (|coef|) versi GA:
- cat__Gender_1: 2.8299
- cat__Gender_2: 2.8041
- cat__comfort_food_chocolate, chips, candy: 1.3178
- cat__comfort_food_Potato chips, ice cream, chocolate, cookies: 1.2487
- cat__comfort_food_Chips, ice cream: 1.2269
- cat__comfort_food_wine. mac and cheese, pizza, ice cream : 1.2149
- cat__comfort_food_peanut butter, dessets, pretzels. : 1.1869
- cat__comfort_food_frozen yogurt, pizza, fast food: 1.1780
- cat__comfort_food_ice cream, cookies, ice cream: 1.1438
- cat__comfort_food_ice cream, cereal, and salt and vinegar chips : -1.1331


In [33]:
# Bagian E — PERBANDINGAN AKHIR & RANGKUM

print("\n==== Perbandingan Akhir (TEST SET) ====")
print(f"OLS -> R^2: {r2_ols:.4f} | MSE: {mse_ols:.4f}")
print(f"GA  -> R^2: {r2_ga:.4f} | MSE: {mse_ga:.4f}")

print("\n[Catatan]")
print("- OLS (Least Squares) memberi solusi analitik optimal untuk SSE pada model linear klasik.")
print("- GA juga meminimalkan SSE (least squares) namun via pencarian evolusioner; hasil bisa mendekati OLS.")
print("- Perbedaan kinerja dapat terjadi karena stochastisitas GA, skala fitur, dan hyperparameter GA.")


==== Perbandingan Akhir (TEST SET) ====
OLS -> R^2: 0.1202 | MSE: 0.3581
GA  -> R^2: 0.0800 | MSE: 0.3745

[Catatan]
- OLS (Least Squares) memberi solusi analitik optimal untuk SSE pada model linear klasik.
- GA juga meminimalkan SSE (least squares) namun via pencarian evolusioner; hasil bisa mendekati OLS.
- Perbedaan kinerja dapat terjadi karena stochastisitas GA, skala fitur, dan hyperparameter GA.
